In [ ]:
from pyDatalog import pyDatalog
import logging
from pyDatalog import pyEngine
pyEngine.Logging = False
logging.basicConfig(level=logging.INFO)
pyDatalog.create_terms('link, can_reach,X,Y,Z')
+link(1,2)
+link(2,3)
+link(2,4)
+link(2,5)
+link(5,6)
+link(6,7)
+link(7,2)
link(X,Y) <= link(Y,X)
can_reach(X,Y) <= link(X,Y)
can_reach(X,Y) <= link(X,Z) & can_reach(Z,Y) & (X!=Y)
print (can_reach(Y,3))

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String, ForeignKey
from ipywidgets import interact_manual

Base = declarative_base(cls=pyDatalog.Mixin, metaclass=pyDatalog.sqlMetaMixin)
engine = create_engine('sqlite:///:memory:', echo=False)

Session = sessionmaker(bind=engine)
session = Session()
Base.session = session

class Person(Base): 
    __tablename__ = 'person'

    name = Column(String, primary_key=True)
    def __init__(self, name): 
        super(Person, self).__init__()
        self.name = name
    
    def __repr__(self):
        return self.name
    
Base.metadata.create_all(engine)
pyDatalog.create_terms('married, bros, a, c, p, X, Y, Z, K')

married(X,Y) <= married(Y,X)
a(X, Y) <= p(X, Y)
a(X, Y) <= p(X, Z) & a(Z, Y)
+p('parent', 'child')


bros(X, Y) <= p(Z, X) & p(Z, Y)
married(X,Y) <= p(X,Z) & p(Y,Z) & (X!=Y) & ~bros(X, Y)# c(Z, X) & c(Z, Y) & (X != Y)

persons = []
def add_person(name):
    for person in persons:
        if person.name == name:
            return person
    print("Created " + name)
    result = Person(name)
    persons.append(result)
    return result

@interact_manual
def get_input(parent='', child=''):
    person1 = add_person(parent)
    person2 = add_person(child)
    if person1 != person2 and not [item for item in p(X,Y) if item[0] == person2 and item[1] == person1] \
        and not [item for item in a(X,Y) if item[0] == person2 and item[1] == person1] \
        and not [item for item in a(X,Y) if item[0] == person1 and item[1] == person2] \
        and not [item for item in (p(Z,X) & p(Z,Y) & (X!=Y)) if item[1] == person1 and item[2] == person2]:
        +p(person1, person2)
    print(married(X,Y))
